In [1]:
import gensim.downloader as api

wv = api.load("word2vec-google-news-300")

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [6]:
wv.similarity(w1="great",w2="good")

0.729151

In [7]:
wv.similarity(w1="profit",w2="loss")

0.34199455

In [8]:
wv.similarity(w1="profit",w2="gain")

0.28772825

In [9]:
wv.most_similar("good")

[('great', 0.7291510105133057),
 ('bad', 0.7190051078796387),
 ('terrific', 0.6889115571975708),
 ('decent', 0.6837348341941833),
 ('nice', 0.6836092472076416),
 ('excellent', 0.644292950630188),
 ('fantastic', 0.6407778263092041),
 ('better', 0.6120728850364685),
 ('solid', 0.5806034803390503),
 ('lousy', 0.576420247554779)]

In [10]:
# KING - MAN + WOMAN = QUEEN

In [12]:
wv.most_similar(positive=["king","woman"],negative=["man"])

[('queen', 0.7118193507194519),
 ('monarch', 0.6189674139022827),
 ('princess', 0.5902431011199951),
 ('crown_prince', 0.5499460697174072),
 ('prince', 0.5377321839332581),
 ('kings', 0.5236844420433044),
 ('Queen_Consort', 0.5235945582389832),
 ('queens', 0.5181134343147278),
 ('sultan', 0.5098593831062317),
 ('monarchy', 0.5087411999702454)]

In [13]:
wv.doesnt_match(["facebook","cat","google","microsoft"])

'cat'

In [14]:
# GLOVE MODEL TWITTER 

In [15]:
glv = api.load("glove-twitter-25")

[==================================================] 100.0% 104.8/104.8MB downloaded


In [16]:
glv.most_similar("good")

[('too', 0.9648017287254333),
 ('day', 0.9533665180206299),
 ('well', 0.9503170847892761),
 ('nice', 0.9438973665237427),
 ('better', 0.9425962567329407),
 ('fun', 0.9418926239013672),
 ('much', 0.9413353800773621),
 ('this', 0.9387555122375488),
 ('hope', 0.9383506774902344),
 ('great', 0.9378516674041748)]

In [17]:
glv.most_similar("tesla")

[('qualcomm', 0.9118991494178772),
 ('motors', 0.907597541809082),
 ('sachs', 0.8855622410774231),
 ('alibaba', 0.8841804265975952),
 ('weber', 0.8666931390762329),
 ('deere', 0.8601743578910828),
 ('kernel', 0.8578172922134399),
 ('foxconn', 0.8552562594413757),
 ('citigroup', 0.8552471399307251),
 ('exxon', 0.855033814907074)]

In [21]:
glv.most_similar("bjp")

[('kejriwal', 0.9592850804328918),
 ('pmln', 0.9451469779014587),
 ('modi', 0.9385530948638916),
 ('arvind', 0.9335629940032959),
 ('congress', 0.9277803897857666),
 ('narendra', 0.9257768392562866),
 ('gujarat', 0.9180455207824707),
 ('anc', 0.9106377959251404),
 ('punjab', 0.9092264175415039),
 ('pml-n', 0.8995295763015747)]

In [23]:
wv_great = wv["great"]
wv_good = wv["good"]

In [35]:
# wv_great

In [20]:
#---------Classification Project-----------------#

In [24]:
import pandas as pd

df = pd.read_csv('fake_and_real_news.csv')
df.shape

(9900, 2)

In [25]:
df.head()

,Text,label
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,Fake
1,U.S. conservative leader optimistic of common ...,Real
2,"Trump proposes U.S. tax overhaul, stirs concer...",Real
3,Court Forces Ohio To Allow Millions Of Illega...,Fake
4,Democrats say Trump agrees to work on immigrat...,Real


In [27]:
df.label.value_counts()

label
Fake    5000
Real    4900
Name: count, dtype: int64

In [28]:
df['label'] = df.label.map({
    'Fake':0,
    'Real':1
})

In [30]:
df.head()

,Text,label
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,0
1,U.S. conservative leader optimistic of common ...,1
2,"Trump proposes U.S. tax overhaul, stirs concer...",1
3,Court Forces Ohio To Allow Millions Of Illega...,0
4,Democrats say Trump agrees to work on immigrat...,1


In [31]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [50]:
def preprocessing_and_vectorize(text):
    doc = nlp(text)
    filtered_token = []
    for token in doc:
        if token.is_stop or token.is_punct:
            continue
        filtered_token.append(token.lemma_)
        
    return wv.get_mean_vector(filtered_token)

In [51]:
# preprocessing_and_vectorize("Don't worry if you don't understand")

In [52]:
# wv.get_mean_vector(["worry","understand"])

In [53]:
df['vector'] = df['Text'].apply(lambda text: preprocessing_and_vectorize(text))

In [54]:
df.head()

,Text,label,vector
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,0,"[0.008657642, 0.019024342, -0.011917442, 0.032..."
1,U.S. conservative leader optimistic of common ...,1,"[0.010864096, 0.007960429, 0.0011915653, 0.014..."
2,"Trump proposes U.S. tax overhaul, stirs concer...",1,"[0.018134918, 0.0062743523, -0.005872244, 0.03..."
3,Court Forces Ohio To Allow Millions Of Illega...,0,"[0.01255197, 0.012613623, 5.9780963e-05, 0.021..."
4,Democrats say Trump agrees to work on immigrat...,1,"[-0.0019059887, 0.011889367, 0.0035395357, 0.0..."


In [55]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(df.vector.values,df.label,test_size=0.2,random_state=2022,stratify=df.label)

In [56]:
X_train.shape

(7920,)

In [57]:
X_test.shape

(1980,)

In [58]:
X_train[:2]

array([array([-2.58219568e-03,  6.66544260e-03,  2.52646767e-03,  3.95730063e-02,
              -1.41182542e-02, -7.22123589e-03,  3.15938070e-02, -3.18326391e-02,
               1.62770562e-02,  2.15391051e-02, -2.36972049e-02, -3.72144543e-02,
              -6.70092739e-03,  1.64224133e-02, -2.48180293e-02,  4.60469536e-02,
               6.02065027e-03,  2.53213309e-02,  4.08343738e-03, -3.32745500e-02,
               6.01468608e-03,  1.92604307e-02,  4.28487025e-02, -2.15403945e-03,
               2.04510838e-02,  3.04712215e-03, -2.73784697e-02, -1.53319014e-03,
               6.67838380e-03, -3.27401888e-03, -1.53824072e-02,  9.19036320e-05,
              -2.74383668e-02, -3.54807038e-04, -1.12002371e-02, -1.66487973e-02,
               1.40579836e-02,  2.69189221e-03, -4.44610231e-03,  3.41382883e-02,
               5.02199680e-03, -1.35964267e-02,  4.29993980e-02,  1.10096289e-02,
              -5.67582203e-03, -1.63680799e-02, -2.08231565e-02,  8.67405720e-03,
              -1

In [59]:
X_train_2d = np.stack(X_train)
X_test_2d = np.stack(X_test)

In [61]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report

In [62]:
clf = GradientBoostingClassifier()
clf.fit(X_train_2d,y_train)

y_pred = clf.predict(X_test_2d)

print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.99      0.97      0.98      1000
           1       0.97      0.99      0.98       980

    accuracy                           0.98      1980
   macro avg       0.98      0.98      0.98      1980
weighted avg       0.98      0.98      0.98      1980

